# Playing with the PAL algorithm.

There's something interesting: https://github.com/VitoChan1/PAL-on-Adder-DSE/blob/master/PAL.py

Summarizing the paper (PAL: An Active Learning Approach to the Multi-Objective Optimization Problem form the Puschel group at ETH).

* error bounds based on hypervolume error
* response surface methods for unevaluated designs. Best so far: PareEGO also uses GP
* scalariztion: without assumptions like convexity not all solutions can be recovered

* uncertainity captured with hyperparameter, there is some scaling parameter beta that determines which fraction of the variance is used. 
* then classified as pareto optimal if the pessimistic bound is not dominated by the optimistic outcome of any other point
* if the optimisitc bound is dominated by the pessimistic bound of any other point, then not pareto optimal

In [1]:
import numpy as np 

In [2]:
def get_pareto_optimal(scores: np.array) -> np.array:
    size = scores.shape[0]
    ids = np.arange(size)
    pareto_front = np.ones(size, dtype=bool)
        for j in range(size):
            # Check if our 'i' point is dominated by our 'j' point
            if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
                # j dominates i. Label 'i' point as dominant
                pareto_front[i] = 0
                break
    # Return ids of pareto front
    return ids[pareto_front]

Let's use PyGMO as they have spent more thought into how to implement this: 

https://esa.github.io/pygmo/documentation/hypervolume.html

In [5]:
import pygmo as pg 

In [13]:
def get_hypervolume(pareto_front: np.array, reference_vector: np.array) -> float: 
    hyp = pg.hypervolume(pareto_front) 
    volume = hyp.compute(reference_vector) # uses 'auto' algorithm
    return volume